In [ ]:
# imports
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

import datetime
import os
import time

from odemis import dataio 
from odemis import model 

ccd = model.getComponent(role='ccd') 


matplotlib.use('TkAgg')

description = "_IFM_30kV_0.23nA_5.3nm_per_s_F4_Bot_RLM"

wdir = "/home/cryo-poc/GDrive/Shared Drives/ENZEL Data 2/Cryo3Beams/Boltje/2024-01-23-Joyce-B-P4/" + datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
wdir += description + "/"
if not os.path.exists(wdir):
    os.makedirs(wdir)

plt.ion() # enable interactive mode matplotlib.pyplot
fig, ax = plt.subplots(dpi=150) # start plotting
x, y1, y2 = [],[],[] # x-axis, mean intensity, max intensity
sc1 = ax.scatter(x,y1)
sc2 = ax.scatter(x,y2)

plt.draw()

hw=3 #half box size in pixels from which the intensity is extracted
i=0 #counter
try:
    while True:
        # update values in array
        im = ccd.data.get() 
        sy, sx = im.shape
        #dataio.tiff.export(wdir + str(i)+".tif", im)
        x.append(np.array(i))
        y1.append(np.array(np.mean(im[sy//2-hw:sy//2+hw,sx//2-hw:sx//2+hw])))
        y2.append(np.array(np.max(im[sy//2-hw:sy//2+hw,sx//2-hw:sx//2+hw])))
        sc1.set_offsets(np.c_[x,y1])
        sc2.set_offsets(np.c_[x,y2])

        # increase counter
        i+=1
        
        # set limits axes plotting
        if len(x) > 100: xmi = x[-100]
        else: xmi = 0
        plt.xlim(xmi,xmi+100) # the x range is from 100 datapoints ago until now
        plt.ylim(np.mean(y1[-50:])*0.8, np.mean(y2[-50:])*1.2) #the y range plotted depends on the previous 50 intensity values

        # render plot
        fig.canvas.draw_idle()
        plt.pause(0.5)
except KeyboardInterrupt:
    print('interrupted!')

plt.waitforbuttonpress()